In [ ]:
!pip install sphinx
!pip install pykeops[full] > install.log
import pykeops
pykeops.clean_pykeops()
import time
import torch
from matplotlib import pyplot as plt
from pykeops.torch import LazyTensor
import numpy as np
from pykeops.torch.cluster import cluster_ranges_centroids
from pykeops.torch.cluster import sort_clusters
from pykeops.torch.cluster import from_matrix
use_cuda = torch.cuda.is_available()
dtype = torch.float32 if use_cuda else torch.float64
if use_cuda:
    torch.cuda.synchronize()

Cleaning /root/.cache/pykeops-1.4.2-cpython-36/...
    - /root/.cache/pykeops-1.4.2-cpython-36/libKeOpstorch1c124fa0f1.cpython-36m-x86_64-linux-gnu.so has been removed.
    - /root/.cache/pykeops-1.4.2-cpython-36/libKeOpstorch52de9bd20c.so has been removed.
    - /root/.cache/pykeops-1.4.2-cpython-36/keops_hash.log has been removed.
    - /root/.cache/pykeops-1.4.2-cpython-36/libKeOpstorch5fba8c75d5.so has been removed.
    - /root/.cache/pykeops-1.4.2-cpython-36/libKeOpstorch5fba8c75d5.cpython-36m-x86_64-linux-gnu.so has been removed.
    - /root/.cache/pykeops-1.4.2-cpython-36/libKeOpstorch52de9bd20c.cpython-36m-x86_64-linux-gnu.so has been removed.
    - /root/.cache/pykeops-1.4.2-cpython-36/libKeOpstorch1c124fa0f1.so has been removed.


In [ ]:
def KMeans(x, K=10, Niter=15, verbose=False):
    """Implements Lloyd's algorithm for the Euclidean metric."""

    start = time.time()
    N, D = x.shape  # Number of samples, dimension of the ambient space

    c = x[:K, :].clone()  # Simplistic initialization for the centroids

    x_i = LazyTensor(x.view(N, 1, D))  # (N, 1, D) samples
    c_j = LazyTensor(c.view(1, K, D))  # (1, K, D) centroids

    # K-means loop:
    # - x  is the (N, D) point cloud,
    # - cl is the (N,) vector of class labels
    # - c  is the (K, D) cloud of cluster centroids
    for i in range(Niter):

        # E step: assign points to the closest cluster -------------------------
        D_ij = ((x_i - c_j) ** 2).sum(-1)  # (N, K) symbolic squared distances
        cl = D_ij.argmin(dim=1).long().view(-1)  # Points -> Nearest cluster

        # M step: update the centroids to the normalized cluster average: ------
        # Compute the sum of points per cluster:
        c.zero_() #sets c to 0
        #scatter_add_(dim,index,src) 
        #https://pytorch-scatter.readthedocs.io/en/1.3.0/functions/add.html
        #adds elements of a source by allocating using a indexing vector
        c.scatter_add_(0, cl[:, None].repeat(1, D), x) #[50,2], obtains the sum of all values that are in the same cluster
        #repeat basically duplicates the whole vector once and concatenates to itself

        # Divide by the number of points per cluster:
        Ncl = torch.bincount(cl, minlength=K).type_as(c).view(K, 1)#bincount gives number of bins per cluster
        c /= Ncl  # in-place division to compute the average

    if verbose:  # Fancy display -----------------------------------------------
        if use_cuda:
            torch.cuda.synchronize()
        end = time.time()
        print(
            f"K-means for the Euclidean metric with {N:,} points in dimension {D:,}, K = {K:,}:"
        )
        print(
            "Timing for {} iterations: {:.5f}s = {} x {:.5f}s\n".format(
                Niter, end - start, Niter, (end - start) / Niter
            )
        )

    return cl, c

def k_argmin(x,y,k=5):
  x_LT=LazyTensor(x.unsqueeze(1))
  y_LT=LazyTensor(y.unsqueeze(0))
  d=((x_LT-y_LT)**2).sum(-1)
  return d.argmin(dim=1).long().view(-1)    

In [ ]:
N, D, K = 10000, 2, 50
x = 0.7 * torch.randn(N, D, dtype=dtype) + 0.3

In [ ]:
#start of the algorithm
if use_cuda:
    torch.cuda.synchronize()

#c = x[:K, :].clone()
cl, c = KMeans(x, K)
cl=k_argmin(x,c) #updating to correct assignment


Compiling libKeOpstorch5fba8c75d5 in /root/.cache/pykeops-1.4.2-cpython-36:
       formula: ArgMin_Reduction(Sum(Square((Var(0,2,0) - Var(1,2,1)))),0)
       aliases: Var(0,2,0); Var(1,2,1); 
       dtype  : float32
... Done.


In [ ]:
if use_cuda:
    torch.cuda.synchronize()
c1=LazyTensor(c.unsqueeze(1)) #c cluster position
c2=LazyTensor(c.unsqueeze(0))
d=((c1-c2)** 2).sum(-1)
#d[a,b]=dist from c[a] to c[b]
ncl=d.argKmin(K=5,dim=1) #top 5 nearest clusters
x_ranges, x_centroids, _ = cluster_ranges_centroids(x, cl)

x, x_labels = sort_clusters(x,cl)
x_LT=LazyTensor(x.unsqueeze(1))

Compiling libKeOpstorch52de9bd20c in /root/.cache/pykeops-1.4.2-cpython-36:
       formula: ArgKMin_Reduction(Sum(Square((Var(0,2,0) - Var(1,2,1)))),5,0)
       aliases: Var(0,2,0); Var(1,2,1); 
       dtype  : float32
... Done.


In [ ]:
#creating mask
r=torch.arange(50).repeat(5,1).T.reshape(-1).long()
keep= torch.zeros([K,K], dtype=torch.bool)          
keep[r,ncl.view(-1)]=True                           
#K=50, 50 x 50

In [ ]:
y = 0.7 * torch.randn(N, D, dtype=dtype) + 0.3
if use_cuda:
    torch.cuda.synchronize()
y_labels=k_argmin(y,c)# 0 3 10 38 ...
y_ranges, y_centroids, _ = cluster_ranges_centroids(y, y_labels)
if use_cuda:
    torch.cuda.synchronize()
y, y_labels = sort_clusters(y, y_labels)    

In [ ]:
if use_cuda:
    torch.cuda.synchronize()
ranges_ij = from_matrix(x_ranges, y_ranges, keep)

y_LT=LazyTensor(y.unsqueeze(0))
D_ij=((y_LT-x_LT)**2).sum(-1)    

In [ ]:
#warmup
D_ij=((y_LT-x_LT)**2).sum(-1)    
D_ij.argKmin(K=5,axis=1)

Compiling libKeOpstorch1c124fa0f1 in /root/.cache/pykeops-1.4.2-cpython-36:
       formula: ArgKMin_Reduction(Sum(Square((Var(0,2,1) - Var(1,2,0)))),5,0)
       aliases: Var(0,2,1); Var(1,2,0); 
       dtype  : float32
... Done.


tensor([[ 291,  298,  240,  293,  274],
        [ 268,  109,  280,    3,   96],
        [ 211,   53,  137,  196,  215],
        ...,
        [9872, 9880, 9953, 9878, 9944],
        [9925, 9895, 9888, 9913, 9962],
        [9927, 9902, 9864, 9904, 9861]])

In [ ]:
#actual timing
brute_force_times=[]
for _ in range(10):
  start=time.time()
  D_ij=((y_LT-x_LT)**2).sum(-1)    
  a=D_ij.argKmin(K=5,axis=1)
  brute_force_times.append(time.time()-start)

print('Time taken for brute force search:',np.mean(brute_force_times))

Time taken for brute force search: 0.003003859519958496


In [ ]:
#testing with reduction
if use_cuda:
    torch.cuda.synchronize()
reduced_times=[]
for _ in range(10):
  start=time.time()
  D_ij=((y_LT-x_LT)**2).sum(-1)  
  D_ij.ranges=ranges_ij
  a_sparse=D_ij.argKmin(K=5,axis=1)
  reduced_times.append(time.time()-start)
print('Time taken with sparse reduction:',np.mean(reduced_times))

Time taken with sparse reduction: 0.0015053033828735351


In [ ]:
reduction=np.mean(reduced_times)/np.mean(brute_force_times)
print('Reduction in time taken:',1-reduction)

Reduction in time taken: 0.4988769039058346


In [ ]:
accuracy=len(torch.nonzero((a==a_sparse).view(-1)))/(N*5)
print("Accuracy:",accuracy)

Accuracy: 0.98172


In [ ]:
#actual from argmin
a

tensor([[ 291,  298,  240,  293,  274],
        [ 268,  109,  280,    3,   96],
        [ 211,   53,  137,  196,  215],
        ...,
        [9872, 9880, 9953, 9878, 9944],
        [9925, 9895, 9888, 9913, 9962],
        [9927, 9902, 9864, 9904, 9861]])

In [ ]:
#with reduction from block sparsity 
a_sparse

tensor([[ 291,  298,  240,  293,  274],
        [ 268,  109,  280,    3,   96],
        [ 211,   53,  137,  196,  215],
        ...,
        [9872, 9880, 9953, 9878, 9944],
        [9925, 9895, 9888, 9913, 9962],
        [9927, 9902, 9864, 9904, 9861]])